# ML Pipeline (Treino e Avaliacao)
Conecta no PostgreSQL, carrega `readings`, treina `RandomForestRegressor` e exibe MAE + gráfico Previsao vs Real.

In [ ]:
import os, pickle
import pandas as pd
import psycopg2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

DB_HOST=os.getenv('DB_HOST','localhost')
DB_PORT=os.getenv('DB_PORT','5432')
DB_NAME=os.getenv('DB_NAME','iot')
DB_USER=os.getenv('DB_USER','iot_user')
DB_PASSWORD=os.getenv('DB_PASSWORD','iot_pass')

conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD)
df = pd.read_sql("SELECT * FROM readings ORDER BY timestamp DESC LIMIT 1000", conn)
conn.close()
df = df.dropna(subset=['temperature','target'])
X = df[['temperature']].values
y = df['target'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)
mae = mean_absolute_error(y_test, pred)
print('MAE:', round(mae,4))
with open('ml/model.pkl','wb') as f:
    pickle.dump(model, f)
plt.figure()
plt.plot(y_test, label='real')
plt.plot(pred, label='pred')
plt.legend(); plt.title('Previsao vs Real'); plt.show()
